In [6]:
# Install necessary packages
%pip install transformers==4.41.2 torch==2.4.0 scikit-learn==1.5.1 pandas==2.2.2 sentence-transformers==2.2.2


In [7]:
import json

# Load the JSON data
with open('translation_evaluation_data.json', 'r') as file:
    evaluation_data = json.load(file)

# Display the first 5 entries to verify data
evaluation_data[:5]


[{'prompt': 'Translate English to French: How are you?',
  'ground_truth': 'Comment Ã§a va?'},
 {'prompt': 'Translate English to French: What is your name?',
  'ground_truth': "Comment tu t'appelles?"},
 {'prompt': 'Translate English to French: Good morning',
  'ground_truth': 'Bonjour'},
 {'prompt': 'Translate English to French: Good night',
  'ground_truth': 'Bonne nuit'},
 {'prompt': 'Translate English to French: Thank you', 'ground_truth': 'Merci'}]

In [9]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Load the quantized model and tokenizer
model_dir = "t5-large"  # Ensure this matches the model you're using
quantized_model_path = "../backend/quantized_t5_large.pth"  # Update with the correct path

# Initialize the tokenizer
tokenizer = T5Tokenizer.from_pretrained(model_dir)

# Initialize the model
model = T5ForConditionalGeneration.from_pretrained(model_dir)

# Load the quantized state dictionary
quantized_state_dict = torch.load(quantized_model_path, map_location=torch.device('cpu'))
model.load_state_dict(quantized_state_dict, strict=False)

# Set the model to evaluation mode
model.eval()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
C:\Users\BarrettFrewA\AppData\Local\Temp\ipykernel_4240\2760224406.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue 

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (d

In [10]:
# Generate translations using the model
for item in evaluation_data:
    input_text = item["prompt"]
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    
    # Generate the translation
    outputs = model.generate(input_ids, max_new_tokens=250, num_beams=5, early_stopping=True)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    
    # Store the generated text
    item["generated_text"] = generated_text

# Convert the data to a DataFrame for easier manipulation
import pandas as pd
df = pd.DataFrame(evaluation_data)


In [11]:
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModel, AutoTokenizer

# Load a pre-trained model for embeddings (e.g., sentence transformers)
embedding_model_name = "sentence-transformers/paraphrase-MiniLM-L6-v2"
embedding_model = AutoModel.from_pretrained(embedding_model_name)
embedding_tokenizer = AutoTokenizer.from_pretrained(embedding_model_name)

def get_embedding(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze()

# Calculate cosine similarity for each pair of ground truth and generated translation
cosine_scores = []
for item in evaluation_data:
    ground_truth_embedding = get_embedding(item["ground_truth"], embedding_model, embedding_tokenizer)
    generated_embedding = get_embedding(item["generated_text"], embedding_model, embedding_tokenizer)
    
    score = cosine_similarity([ground_truth_embedding], [generated_embedding])[0][0]
    item["cosine_similarity"] = score

# Convert the data into a DataFrame
df = pd.DataFrame(evaluation_data)


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

c:\Users\BarrettFrewA\Jupyter Notebooks\ABFLLMchatbot\LLM\.venv\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\BarrettFrewA\.cache\huggingface\hub\models--sentence-transformers--paraphrase-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [16]:
%pip install openpyxl

import pandas as pd

# Assuming you've already run the previous cells where you calculated cosine similarity and created the DataFrame `df`.
# Calculate average cosine similarity
average_cosine_similarity = df["cosine_similarity"].mean()
print(f"Average Cosine Similarity: {average_cosine_similarity:.4f}")

# Save as Excel
df.to_excel('evaluation_results.xlsx', index=False)
print("Data saved as evaluation_results.xlsx")

df



  Using cached et_xmlfile-1.1.0-py3-none-any.whl.metadata (1.8 kB)
Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.
Average Cosine Similarity: 0.7984
Data saved as evaluation_results.xlsx


,prompt,ground_truth,generated_text,cosine_similarity
0,Translate English to French: How are you?,Comment Ã§a va?,Comment en êtes-vous?,0.536611
1,Translate English to French: What is your name?,Comment tu t'appelles?,Quel est votre nom?,0.435010
2,Translate English to French: Good morning,Bonjour,Bonjour,1.000000
3,Translate English to French: Good night,Bonne nuit,Bonne nuit,1.000000
4,Translate English to French: Thank you,Merci,Merci.,0.973977
...,...,...,...,...
91,Translate English to French: I would like to b...,Je voudrais acheter Ã§a,J'aimerais acheter,0.266523
92,Translate English to French: Can you help me w...,Pouvez-vous m'aider avec mes bagages?,Pouvez-vous m'aider avec mes bagages?,1.000000
93,Translate English to French: I would like to c...,Je voudrais m'enregistrer,J'aimerais m'inscrire,0.340102
94,Translate English to French: What time does th...,Ã€ quelle heure part le train?,quelle heure le train part-il?,0.863919
